# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk import word_tokenize, sent_tokenize
import re
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet') 
nltk.download('punkt')
nltk.download('punkt_tab')
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import numpy as np
from sklearn.model_selection import GridSearchCV
from joblib import parallel_backend
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\semer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\semer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\semer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\semer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
with engine.connect() as conn, conn.begin():  
    df = pd.read_sql_table("CleanData", conn) 
X = df['message']
Y = df[df.columns.difference(['id', 'message', 'original', 'genre'])]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = text.lower().strip()
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    cleanedtokens = [WordNetLemmatizer().lemmatize(w) for w in words]
    return cleanedtokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline =  Pipeline([
    ('features',CountVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
with parallel_backend('threading', n_jobs=4):
    X_train, X_test, y_train, y_test = train_test_split(X, Y)
    pipeline.fit(X_train, y_train)
pickle.dump(pipeline, open('baseline.pkl', 'wb'))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred_np = np.array(y_pred)
y_test_np = y_test.values
for i in range(y_pred_np.shape[1]):
    print(classification_report(y_test_np[:,i], y_pred_np[:,i]))

In [ ]:
pipeline.get_params().keys()

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline_cv =  Pipeline([
    ('features',CountVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

parameters = {
        'features__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [10, 50, 100]
    }
cv = GridSearchCV(pipeline_cv,  param_grid=parameters)

In [ ]:
with parallel_backend('threading', n_jobs=4):
    cv.fit(X_train, y_train)

In [ ]:
pickle.dump(cv, open('cv.pkl', 'wb'))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_cv = cv.predict(X_test)
y_pred_cv_np = np.array(y_pred_cv)
for i in range(y_pred_np.shape[1]):
    print(classification_report(y_test_np[:,i], y_pred_cv_np[:,i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_updated =  Pipeline([
    ('features',CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=50)))
])

In [ ]:
parameters = {
        'features__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [10, 50, 100]
    }
cv_updated = GridSearchCV(pipeline_updated,  param_grid=parameters)

In [ ]:
with parallel_backend('threading', n_jobs=4):
    cv_updated.fit(X_train, y_train)

In [ ]:
y_pred_pu = cv_updated.predict(X_test)
y_pred_pu_np = np.array(y_pred_pu)
for i in range(y_pred_np.shape[1]):
    print(classification_report(y_test_np[:,i], y_pred_pu_np[:,i]))

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv_updated, open('cv_updated.pkl', 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.